In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import concurrent.futures
from joblib import Parallel, delayed
import multiprocessing
from sklearn.cluster import DBSCAN
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import seaborn as sns

In [2]:
ratings = pd.read_csv('ratings.csv')
to_read = pd.read_csv('to_read.csv')
books = pd.read_csv('books.csv')
book_tags = pd.read_csv('book_tags.csv')
tags = pd.read_csv('tags.csv')

In [3]:
def explore_data(df):
    print(df.head())

In [4]:
explore_data(ratings)
explore_data(to_read)
explore_data(books)
explore_data(book_tags)
explore_data(tags)

   user_id  book_id  rating
0        1      258       5
1        2     4081       4
2        2      260       5
3        2     9296       5
4        2     2318       3
   user_id  book_id
0        9        8
1       15      398
2       15      275
3       37     7173
4       34      380
   book_id  goodreads_book_id  best_book_id  work_id  books_count       isbn  \
0        1            2767052       2767052  2792775          272  439023483   
1        2                  3             3  4640799          491  439554934   
2        3              41865         41865  3212258          226  316015849   
3        4               2657          2657  3275794          487   61120081   
4        5               4671          4671   245494         1356  743273567   

         isbn13                      authors  original_publication_year  \
0  9.780439e+12              Suzanne Collins                     2008.0   
1  9.780440e+12  J.K. Rowling, Mary GrandPré                     1997.0   
2  9.7

In [6]:
import numpy as np
import pandas as pd
import random
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Завантаження ваших даних
# Припустимо, у нас є DataFrame `ratings_df` з вашими рейтингами

# Агрегація даних (наприклад, середній рейтинг на користувача)
user_features = ratings.groupby('user_id').agg({'rating': ['mean', 'count']})

# Нормалізація даних
scaler = StandardScaler()
user_features_scaled = scaler.fit_transform(user_features)

# Кластеризація
kmeans = KMeans(n_clusters=7)  # Припустимо, ми вибрали 5 кластерів
user_clusters = kmeans.fit_predict(user_features_scaled)

# Додавання інформації про кластери до даних користувачів
user_features['cluster'] = user_clusters

# Ініціалізація Q-таблиці
n_clusters = 7  # кількість кластерів
n_books = 10000   # припустимо, у нас є 100 книг
Q = np.zeros((n_clusters, n_books))

# Параметри Q-learning
alpha = 0.1   # швидкість навчання
gamma = 0.6   # коефіцієнт дисконтування
epsilon = 0.1 # ймовірність дослідження

# Припустима функція для отримання винагороди
def get_reward(cluster, book):
    # Припустимо, винагорода визначається довільно
    return random.randint(0, 1)

# Процес навчання
for episode in range(1, 1001):
    state = np.random.randint(0, n_clusters)  # вибір випадкового кластера (стану)
    
    if random.uniform(0, 1) < epsilon:
        action = np.random.randint(0, n_books)  # випадкова дія (рекомендація книги)
    else:
        action = np.argmax(Q[state, :])  # найкраща дія згідно Q-таблиці

    # Отримання винагороди
    reward = get_reward(state, action)

    # Оновлення Q-таблиці
    old_value = Q[state, action]
    next_max = np.max(Q[state, :])
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
    Q[state, action] = new_value

# Виведення оновленої Q-таблиці
print(Q)

C:\Users\Andrew\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[[1.27019957 0.         0.         ... 0.         0.         0.        ]
 [1.1429826  0.         0.         ... 0.         0.         0.        ]
 [1.21330245 0.         0.         ... 0.         0.         0.        ]
 ...
 [1.23235851 0.         0.         ... 0.         0.         0.        ]
 [1.4084166  0.         0.         ... 0.         0.         0.        ]
 [1.13903626 0.         0.         ... 0.         0.         0.        ]]
Рекомендовані книги: [ 848  959 8875 6184 9775 4646 1170 3161 4952    0]


C:\Users\Andrew\AppData\Local\Temp\ipykernel_46208\1433401342.py:62: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  recommended_books = np.argsort(Q[int(user_cluster), :])[-10:]  # Топ-10 книг


In [12]:
# Функція для генерації рекомендацій
def recommend_books(user_cluster):
    recommended_books = np.argsort(Q[int(user_cluster), :])[-10:]  # Топ-10 книг
    return recommended_books

# Припустимо, ми отримуємо кластер для користувача
user_cluster = user_features.loc[345, 'cluster']
recommended_books = recommend_books(user_cluster)

print("Рекомендовані книги:", ','.join(recommended_books.astype('str')))


Рекомендовані книги: 848,959,8875,6184,9775,4646,1170,3161,4952,0


C:\Users\Andrew\AppData\Local\Temp\ipykernel_46208\1010859453.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  recommended_books = np.argsort(Q[int(user_cluster), :])[-10:]  # Топ-10 книг


In [15]:
books[books['book_id'].isin([848,959,8875,6184,9775,4646,1170,3161,4952,0])]['title']

847                             The Bonesetter's Daughter
958               A Portrait of the Artist as a Young Man
1169                          Unravel Me (Shatter Me, #2)
3160                                           Stay Close
4645    Chicken Soup with Rice: A Book of Months (The ...
4951            Midnight (Warriors: The New Prophecy, #1)
6183                                                Money
8874    What My Mother Doesn't Know (What My Mother Do...
9774         Sweet Tooth, Volume 1: Out of the Deep Woods
Name: title, dtype: object

In [23]:
#Симуляція 
import numpy as np
import pandas as pd
import random

# Кількість користувачів і книг
num_users = 100
num_books = 1000

# Створення симульованих даних
np.random.seed(42)
simulated_data = {
    "user_id": [],
    "liked_books": []
}

for user_id in range(num_users):
    # Припустимо, кожен користувач сподобав 10 книг
    liked_books = random.sample(range(num_books), 10)
    simulated_data["user_id"].append(user_id)
    simulated_data["liked_books"].append(liked_books)

# Перетворення в DataFrame
simulated_df = pd.DataFrame(simulated_data)

# Перегляд деяких даних
print(simulated_df.head())

   user_id                                        liked_books
0        0  [553, 552, 79, 960, 500, 636, 696, 492, 189, 121]
1        1  [516, 354, 782, 998, 213, 852, 49, 427, 176, 732]
2        2    [585, 60, 361, 65, 32, 141, 907, 137, 945, 403]
3        3  [912, 963, 750, 873, 402, 859, 304, 770, 976, ...
4        4   [819, 84, 601, 279, 15, 690, 715, 686, 397, 298]


In [22]:
import numpy as np

# Функція для розрахунку HR@K
def hit_ratio(recommended_items, actual_items, k):
    top_k_items = set(recommended_items[:k])
    hits = len(top_k_items.intersection(actual_items))
    return hits / len(actual_items) if actual_items else 0

# Функція для розрахунку NDCG@K
def ndcg(recommended_items, actual_items, k):
    top_k_items = recommended_items[:k]
    dcg = 0
    idcg = sum([1.0 / np.log(i + 2) for i in range(min(len(actual_items), k))])
    for i, item in enumerate(top_k_items):
        if item in actual_items:
            dcg += 1.0 / np.log(i + 2)
    return dcg / idcg if idcg > 0 else 0

# Приклад даних
actual_items_user = [2818, 1126, 4519] # Фактичні книги, які сподобалися користувачу
recommended_books = [848,959,8875,6184,9775,4646,1170,3161,4952] # Рекомендовані книги

# Розрахунок метрик
k = 5
hr_at_k = hit_ratio(recommended_books, actual_items_user, k)
ndcg_at_k = ndcg(recommended_books, actual_items_user, k)

print("HR@K:", hr_at_k)
print("NDCG@K:", ndcg_at_k)

HR@K: 0.0
NDCG@K: 0.0


In [18]:
# Функції для розрахунку HR@K та NDCG@K вже були надані раніше

# Припустимо, у нас є список рекомендацій для кожного користувача
# Тут ми використовуємо випадкові рекомендації для прикладу
np.random.seed(42)
recommendations = {user: random.sample(range(num_books), 10) for user in range(num_users)}

# Розрахунок метрик
k = 5
hr_scores = []
ndcg_scores = []

for user_id, actual_books in simulated_df.iterrows():
    recommended_books = recommendations[user_id]
    actual_books = actual_books["liked_books"]

    hr_scores.append(hit_ratio(recommended_books, actual_books, k))
    ndcg_scores.append(ndcg(recommended_books, actual_books, k))

# Обчислення середніх значень
mean_hr = np.mean(hr_scores)
mean_ndcg = np.mean(ndcg_scores)

print("Середній HR@K:", mean_hr)
print("Середній NDCG@K:", mean_ndcg)


Середній HR@K: 0.004
Середній NDCG@K: 0.010235117527941014


In [36]:
import numpy as np
import pandas as pd
import random

# Function to simulate user preferences
def simulate_user_preferences(num_users, num_books, books_per_user):
    user_prefs = {}
    for user in range(num_users):
        liked_books = random.sample(range(num_books), books_per_user)
        user_prefs[user] = liked_books
    return user_prefs

# Function to generate recommendations (placeholder)
def generate_recommendations(user, num_books):
    return random.sample(range(num_books), 10)

# Functions for HR@K and NDCG@K (defined previously)

# Simulate user preferences
num_users = 6
num_books = 100
books_per_user = 3
user_preferences = simulate_user_preferences(num_users, num_books, books_per_user)

# Evaluate recommendations
k = 10
hr_scores = []
ndcg_scores = []

for user in range(num_users):
    actual_liked_books = user_preferences[user]
    recommended_books = generate_recommendations(user, num_books)
    hr_scores.append(hit_ratio(recommended_books, actual_liked_books, k))
    ndcg_scores.append(ndcg(recommended_books, actual_liked_books, k))

# Calculate average scores
average_hr = np.mean(hr_scores)
average_ndcg = np.mean(ndcg_scores)

print(f"Average HR@{k}: {average_hr}")
print(f"Average NDCG@{k}: {average_ndcg}")

Average HR@10: 0.1111111111111111
Average NDCG@10: 0.05811704125059249


In [56]:
import pandas as pd

# Основна функція

# Припустимо, у вас є файли 'ratings.csv', 'books.csv', 'tags.csv', 'book_tags.csv'
ratings = pd.read_csv('ratings.csv')
books = pd.read_csv('books.csv')
book_tags = pd.read_csv('book_tags.csv')
tags = pd.read_csv('tags.csv')

# Попередня обробка даних
# Наприклад, видалення непотрібних стовпців
books = books[['book_id', 'authors', 'original_publication_year', 'title', 'average_rating']]
book_tags['book_id'] = book_tags['goodreads_book_id']

# Об'єднання даних про книги з тегами
books_with_tags = pd.merge(books, book_tags, on='book_id')
books_with_tags = pd.merge(books_with_tags, tags, on='tag_id')

# Виведення декількох перших рядків для перевірки
print(books_with_tags.head())




   book_id                      authors  original_publication_year  \
0        1              Suzanne Collins                     2008.0   
1        2  J.K. Rowling, Mary GrandPré                     1997.0   
2        3              Stephenie Meyer                     2005.0   
3        5          F. Scott Fitzgerald                     1925.0   
4        6                   John Green                     2012.0   

                                               title  average_rating  \
0            The Hunger Games (The Hunger Games, #1)            4.34   
1  Harry Potter and the Sorcerer's Stone (Harry P...            4.44   
2                            Twilight (Twilight, #1)            3.57   
3                                   The Great Gatsby            3.89   
4                             The Fault in Our Stars            4.26   

   goodreads_book_id  tag_id   count tag_name  
0                  1   30574  167697  to-read  
1                  2   30574   24549  to-read  
2 

In [60]:
def perform_clustering(data):
    # Перетворення жанрів у числові значення за допомогою one-hot encoding
    genres_dummies = data['genres'].str.get_dummies(sep=',')
    data = pd.concat([data, genres_dummies], axis=1)

    # Використання середнього рейтингу та жанрів для кластеризації
    features = data[['average_rating'] + list(genres_dummies.columns)]

    # Нормалізація даних
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Використання K-середніх для кластеризації
    kmeans = KMeans(n_clusters=7, random_state=42)
    kmeans.fit(scaled_features)
    data['cluster'] = kmeans.labels_

    return data

In [63]:
books_with_tags

# Перетворення жанрів у рядок (якщо вони ще не у такому форматі)
books_with_tags['genres'] = books_with_tags['tag_name'].apply(lambda x: ','.join(x))

# Виконання кластеризації
clustered_books = perform_clustering(books_with_tags)

# Виведення результатів
print(clustered_books.head())

C:\Users\Andrew\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


   book_id                      authors  original_publication_year  \
0        1              Suzanne Collins                     2008.0   
1        2  J.K. Rowling, Mary GrandPré                     1997.0   
2        3              Stephenie Meyer                     2005.0   
3        5          F. Scott Fitzgerald                     1925.0   
4        6                   John Green                     2012.0   

                                               title  average_rating  \
0            The Hunger Games (The Hunger Games, #1)            4.34   
1  Harry Potter and the Sorcerer's Stone (Harry P...            4.44   
2                            Twilight (Twilight, #1)            3.57   
3                                   The Great Gatsby            3.89   
4                             The Fault in Our Stars            4.26   

   goodreads_book_id  tag_id   count tag_name         genres  ...  Ｆ  ａ  ｅ  ｉ  \
0                  1   30574  167697  to-read  t,o,-,r,e,a,d  ...

In [62]:
clustered_books

,book_id,authors,original_publication_year,title,average_rating,goodreads_book_id,tag_id,count,tag_name,genres,...,Ｆ,ａ,ｅ,ｉ,ｏ,ｒ,ｓ,ｔ,ｖ,cluster
0,1,Suzanne Collins,2008.0,"The Hunger Games (The Hunger Games, #1)",4.34,1,30574,167697,to-read,"t,o,-,r,e,a,d",...,0,0,0,0,0,0,0,0,0,0
1,2,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Sorcerer's Stone (Harry P...,4.44,2,30574,24549,to-read,"t,o,-,r,e,a,d",...,0,0,0,0,0,0,0,0,0,0
2,3,Stephenie Meyer,2005.0,"Twilight (Twilight, #1)",3.57,3,30574,496107,to-read,"t,o,-,r,e,a,d",...,0,0,0,0,0,0,0,0,0,0
3,5,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,5,30574,11909,to-read,"t,o,-,r,e,a,d",...,0,0,0,0,0,0,0,0,0,0
4,6,John Green,2012.0,The Fault in Our Stars,4.26,6,30574,298,to-read,"t,o,-,r,e,a,d",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81195,9998,Patrick O'Brian,1977.0,The Mauritius Command,4.35,9998,21601,5,nippon,"n,i,p,p,o,n",...,0,0,0,0,0,0,0,0,0,2
81196,9998,Patrick O'Brian,1977.0,The Mauritius Command,4.35,9998,34241,5,日本語,"日,本,語",...,0,0,0,0,0,0,0,0,0,2
81197,9998,Patrick O'Brian,1977.0,The Mauritius Command,4.35,9998,16220,5,japanese-novels,"j,a,p,a,n,e,s,e,-,n,o,v,e,l,s",...,0,0,0,0,0,0,0,0,0,0
81198,9998,Patrick O'Brian,1977.0,The Mauritius Command,4.35,9998,11793,4,fiction-japan,"f,i,c,t,i,o,n,-,j,a,p,a,n",...,0,0,0,0,0,0,0,0,0,0


In [67]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

def create_user_item_matrix(data):
    # Створення матриці користувач-книга з оцінками
    user_item_matrix = data.pivot_table(index='user_id', columns='book_id', values='rating').fillna(0)
    return user_item_matrix

def recommend_books(user_id, user_item_matrix, n_recommendations=5):
    # Перетворення матриці у формат CSR для ефективності
    user_item_csr = csr_matrix(user_item_matrix.values)

    # Обчислення косинусної схожості між користувачами
    user_similarity = cosine_similarity(user_item_csr)
    user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

    # Отримання схожих користувачів
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).iloc[1:].head(n_recommendations)
    similar_users_ids = similar_users.index

    # Збір рекомендацій з схожих користувачів
    recommended_books = []
    for similar_user in similar_users_ids:
        max_rating_books = user_item_matrix.loc[similar_user].sort_values(ascending=False).head(n_recommendations).index
        recommended_books.extend(max_rating_books)

    return list(set(recommended_books))

In [68]:
# Припустимо, у вас є дані про оцінки користувачів

# Створення матриці користувач-книга
user_item_matrix = create_user_item_matrix(ratings)

# Рекомендація книг для конкретного користувача
user_id = 1  # Приклад ID користувача
recommended_books = recommend_books(user_id, user_item_matrix)

# Виведення рекомендованих книг
print("Рекомендовані книги:", recommended_books)

MemoryError: Unable to allocate 21.3 GiB for an array with shape (53424, 53424) and data type float64